In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

import torch
from ptflops import get_model_complexity_info

# 参数设置

# 训练参数 
除了路径 其他不要变

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"E:\TECO_Works\Conference\ISWC2022\Run_logs"              
args.freq_save_path   = r"E:\TECO_Works\Conference\ISWC2022\Freq_data"
args.window_save_path = r"E:\TECO_Works\Conference\ISWC2022\Sliding_window"
args.root_path        = r"E:\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## 数据参数



In [3]:
# oppo : 25/50  2
# dg   : 12/24  4
# dsads: 12/24  4
# pamap2: 10/20  5
# wisdm: 12/24  4
# skodar : 12/24 4

In [4]:
args.seed                             = 1


args.data_name                        =  "pamap2"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 12


args.difference       = False 
args.filtering        =  False
args.magnitude        =  False
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]


if args.difference:
    args.c_in = args.c_in*2

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



## 模型参数

In [5]:
# dsads 27838817.0 99367   26
# skodar  9907786.0   67092 24
# wisdm 733382.0 16
# dg  1041602.0 16
# oppo 7377290.0 28

In [6]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 37274
Set the seed as :  1


In [ ]:
exp.

In [7]:
macs, params = get_model_complexity_info(exp.model, (1  ,args.input_length,args.c_in), as_strings=False,
                                       print_per_layer_stat=True, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  37.27 k, 99.995% Params, 9.56 MMac, 100.000% MACs, 
  (model): TinyHAR_Model(
    37.27 k, 99.995% Params, 9.56 MMac, 100.000% MACs, 
    (layers_conv): ModuleList(
      6.34 k, 17.009% Params, 7.72 MMac, 80.721% MACs, 
      (0): Sequential(
        160, 0.429% Params, 531.36 KMac, 5.556% MACs, 
        (0): Conv2d(120, 0.322% Params, 354.24 KMac, 3.704% MACs, 1, 20, kernel_size=(5, 1), stride=(1, 1))
        (1): ReLU(0, 0.000% Params, 59.04 KMac, 0.617% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.107% Params, 118.08 KMac, 1.235% MACs, 20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        2.06 k, 5.527% Params, 3.0 MMac, 31.319% MACs, 
        (0): Conv2d(2.02 k, 5.419% Params, 2.91 MMac, 30.415% MACs, 20, 20, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(0, 0.000% Params, 28.8 KMac, 0.301% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.107% Params, 57.6 KMac, 0.602% MACs, 20, eps=1e-05, momentum

In [71]:
args.filter_scaling_factor =1
args.model_type              = "mcnn"#"deepconvlstm"#"sahar" #"deepconvlstm"  "deepconvlstm_attn"
exp = Exp(args)
#exp.model

Use GPU: cuda:0
clone the  wavefiler weight
Build the Milti-Branch CNN model!
Done!
Parameter : 574558
Set the seed as :  1


In [72]:



macs, params = get_model_complexity_info(exp.model, (1  ,args.input_length,args.c_in), as_strings=False,
                                       print_per_layer_stat=True, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  574.56 k, 100.000% Params, 106.16 MMac, 100.000% MACs, 
  (wave_conv): Wavelet_learnable_filter(
    4.12 k, 0.716% Params, 10.5 MMac, 9.887% MACs, 
    (Filter_ReplicationPad2d): ReflectionPad2d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, (42, 42, 0, 0))
    (wavelet_conv): Conv2d(4.12 k, 0.716% Params, 10.5 MMac, 9.887% MACs, 1, 49, kernel_size=(1, 84), stride=(1, 1), padding=valid, bias=False)
  )
  (model): MCNN(
    570.44 k, 99.284% Params, 95.66 MMac, 90.113% MACs, 
    (conv_blocks): ModuleList(
      77.38 k, 13.467% Params, 91.73 MMac, 86.406% MACs, 
      (0): ConvBlock(
        36.29 k, 6.316% Params, 61.43 MMac, 57.869% MACs, 
        (conv1): Conv2d(15.74 k, 2.740% Params, 37.79 MMac, 35.594% MACs, 49, 64, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(0, 0.000% Params, 226.56 KMac, 0.213% MACs, inplace=True)
        (conv2): Conv2d(20.54 k, 3.576% Params, 23.42 MMac, 22.062% MACs, 64, 64, kernel_size=(5, 1), stride=(2, 1))
      )
      (1): ConvBlo

In [5]:
# cross_channel_interaction_type = "attn",    # attn  transformer  identity
# cross_channel_aggregation_type = "filter",  # filter  naive  FC  "SFCC", "SFCF"
# temporal_info_interaction_type = "gru",     # gru  lstm  attn  transformer  identity  conv
# temporal_info_aggregation_type = "FC",      # naive  filter  FC  tnaive

# 实验

In [8]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 370082
Set the seed as :  1


In [5]:
exp.model

model_builder(
  (model): MCNN(
    (conv_blocks): ModuleList(
      (0): ConvBlock(
        (conv1): Conv2d(1, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
      (1): ConvBlock(
        (conv1): Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (activation): ReLU()
    (fc_sensor_fusion): Linear(in_features=48, out_features=32, bias=True)
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc_temporal_fusion): Linear(in_features=608, out_features=32, bias=True)
    (fc_prediction): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [6]:
exp.train()

 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.0001465  0.000181   0.00097182 0.00123001 0.0004852  0.00060716]
Train data number :  17901
The number of classes is :  6
The input_length  is :  100
The channel_in is :  3
Validation data number :  1989
Test data number :  9947
================ Build the model ================ 
Build the Milti-Branch CNN model!
Epoch: 1 cost time: 6.765257120132446
VALI: Epoch: 1, Steps: 70 | Train Loss: 1.3114785  Vali Loss: 0.8842771 Vali Accuracy: 0.6596280  Vali weighted F1: 0.5520466  Vali macro F1 0.2689652 
Validation loss decreased (inf --> 0.884277).  Sav

Epoch: 27 cost time: 5.217180967330933
VALI: Epoch: 27, Steps: 70 | Train Loss: 0.1504892  Vali Loss: 0.1550815 Vali Accuracy: 0.9451986  Vali weighted F1: 0.9450059  Vali macro F1 0.9224148 
new best score!!!!
Validation loss decreased (0.166138 --> 0.155081).  Saving model ...
new best score!!!!
Epoch: 28 cost time: 5.159158945083618
VALI: Epoch: 28, Steps: 70 | Train Loss: 0.1477792  Vali Loss: 0.1937707 Vali Accuracy: 0.9351433  Vali weighted F1: 0.9329186  Vali macro F1 0.9108405 
EarlyStopping counter: 1 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 29 cost time: 5.173162221908569
VALI: Epoch: 29, Steps: 70 | Train Loss: 0.1442873  Vali Loss: 0.1477436 Vali Accuracy: 0.9487179  Vali weighted F1: 0.9487887  Vali macro F1 0.9243086 
new best score!!!!
Validation loss decreased (0.155081 --> 0.147744).  Saving model ...
new best score!!!!
Epoch: 30 cost time: 5.1555047035217285
VALI: Epoch: 30, Steps: 70 | Train Loss: 0.1415625  Vali Loss: 0.1614549 Vali Accuracy: 0.9

KeyboardInterrupt: 

In [ ]:
dataset = data_dict[args.data_name](args)

In [ ]:
import numpy as np

In [ ]:
start_index = dataset.train_slidingwindows[0][1]
end_index   = dataset.train_slidingwindows[0][2]
sample_x_1    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

start_index = dataset.train_slidingwindows[100][1]
end_index   = dataset.train_slidingwindows[100][2]
sample_x_2    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

temp_1 = np.expand_dims(sample_x_1,0)
temp_2 = np.expand_dims(sample_x_2,0)
combined_x = np.concatenate([temp_1,temp_2],axis=0)
combined_x = np.expand_dims(combined_x,1)
combined_x = torch.tensor(combined_x).double().to(exp.device)
print(combined_x.shape)

In [ ]:
out = exp.model(combined_x)
out = out.cpu().detach().numpy()

In [ ]:
out.shape

In [ ]:
weight = exp.model.wave_conv.wavelet_conv.weight.cpu().detach().numpy()

In [ ]:
for n,p in exp.model.named_parameters():
    print(n)

In [ ]:
import matplotlib.pyplot as plt
index1 = 0
index2 = 0

plt.figure()
plt.plot(out[0,0,:,index2])
plt.plot(out[0,1,:,index2])
plt.plot(out[0,2,:,index2])
plt.plot(out[0,3,:,index2])

In [ ]:
# 如果我们设置为 
# args.wavelet_filtering_learnable      = True
# exp = Exp(args)